# Lab | Data Aggregation and Filtering

In this challenge, we will continue to work with customer data from an insurance company. We will use the dataset called marketing_customer_analysis.csv, which can be found at the following link:

https://raw.githubusercontent.com/data-bootcamp-v4/data/main/marketing_customer_analysis.csv

This dataset contains information such as customer demographics, policy details, vehicle information, and the customer's response to the last marketing campaign. Our goal is to explore and analyze this data by first performing data cleaning, formatting, and structuring.

In [11]:
import pandas as pd

marketing_data_url = "https://raw.githubusercontent.com/data-bootcamp-v4/data/main/marketing_customer_analysis.csv"
marketing_df = pd.read_csv(marketing_data_url)
marketing_df.columns = marketing_df.columns.str.lower().str.replace(" ", "_")

1. Create a new DataFrame that only includes customers who have a total_claim_amount greater than $1,000 and have a response of "Yes" to the last marketing campaign.

In [12]:
high_claim_responders = marketing_df[(marketing_df['total_claim_amount'] > 1000) & (marketing_df['response'] == 'Yes')]
print("High Claim Responders:")
print(high_claim_responders.head())

High Claim Responders:
     unnamed:_0 customer       state  customer_lifetime_value response  \
189         189  OK31456  California             11009.130490      Yes   
236         236  YJ16163      Oregon             11009.130490      Yes   
419         419  GW43195      Oregon             25807.063000      Yes   
442         442  IP94270     Arizona             13736.132500      Yes   
587         587  FJ28407  California              5619.689084      Yes   

     coverage             education effective_to_date employmentstatus gender  \
189   Premium              Bachelor           1/24/11         Employed      F   
236   Premium              Bachelor           1/24/11         Employed      F   
419  Extended               College           2/13/11         Employed      F   
442   Premium                Master           2/13/11         Disabled      F   
587   Premium  High School or Below           1/26/11       Unemployed      M   

     ...  number_of_open_complaints number_of

2. Using the original Dataframe, analyze the average total_claim_amount by each policy type and gender for customers who have responded "Yes" to the last marketing campaign. Write your conclusions.

In [3]:
yes_responders = marketing_df[marketing_df['response'] == 'Yes']
avg_claim_pivot = yes_responders.pivot_table(index=['policy_type', 'gender'], values='total_claim_amount', aggfunc='mean').round(2)
print("Average Total Claim Amount by Policy Type and Gender:")
print(avg_claim_pivot)
# Corporate policies tend to have the highest average claim amounts. In most policy types, male customers tend to have slightly higher average claim amounts than female customers. 

Average Total Claim Amount by Policy Type and Gender:
                       total_claim_amount
policy_type    gender                    
Corporate Auto F                   433.74
               M                   408.58
Personal Auto  F                   452.97
               M                   457.01
Special Auto   F                   453.28
               M                   429.53


3. Analyze the total number of customers who have policies in each state, and then filter the results to only include states where there are more than 500 customers.

In [14]:
state_counts = marketing_df['state'].value_counts()
states_over_500 = state_counts[state_counts > 500]
print("States with more than 500 customers:")
print(states_over_500)

States with more than 500 customers:
state
California    3552
Oregon        2909
Arizona       1937
Nevada         993
Washington     888
Name: count, dtype: int64


4. Find the maximum, minimum, and median customer lifetime value by education level and gender. Write your conclusions.

In [15]:
clv_stats = marketing_df.groupby(['education', 'gender'])['customer_lifetime_value'].agg(['max', 'min', 'median']).round(2)
print("Customer Lifetime Value Statistics:")
print(clv_stats)

Customer Lifetime Value Statistics:
                                  max      min   median
education            gender                            
Bachelor             F       73225.96  1904.00  5640.51
                     M       67907.27  1898.01  5548.03
College              F       61850.19  1898.68  5623.61
                     M       61134.68  1918.12  6005.85
Doctor               F       44856.11  2395.57  5332.46
                     M       32677.34  2267.60  5577.67
High School or Below F       55277.45  2144.92  6039.55
                     M       83325.38  1940.98  6286.73
Master               F       51016.07  2417.78  5729.86
                     M       50568.26  2272.31  5579.10


## Bonus

5. The marketing team wants to analyze the number of policies sold by state and month. Present the data in a table where the months are arranged as columns and the states are arranged as rows.

In [16]:
marketing_df['effective_to_date'] = pd.to_datetime(marketing_df['effective_to_date'])
marketing_df['month'] = marketing_df['effective_to_date'].dt.month
policies_by_state_month = marketing_df.pivot_table(index='state', columns='month', values='policy', aggfunc='count', fill_value=0)
print("Policies Sold by State and Month:")
print(policies_by_state_month)

Policies Sold by State and Month:
month          1     2
state                 
Arizona     1008   929
California  1918  1634
Nevada       551   442
Oregon      1565  1344
Washington   463   425


/var/folders/c6/hdyn7chd4w10061yk0b90r8w0000gn/T/ipykernel_80518/2626085914.py:1: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  marketing_df['effective_to_date'] = pd.to_datetime(marketing_df['effective_to_date'])


6.  Display a new DataFrame that contains the number of policies sold by month, by state, for the top 3 states with the highest number of policies sold.

*Hint:*
- *To accomplish this, you will first need to group the data by state and month, then count the number of policies sold for each group. Afterwards, you will need to sort the data by the count of policies sold in descending order.*
- *Next, you will select the top 3 states with the highest number of policies sold.*
- *Finally, you will create a new DataFrame that contains the number of policies sold by month for each of the top 3 states.*

In [7]:
top_states = marketing_df['state'].value_counts().nlargest(3).index
top_states_df = marketing_df[marketing_df['state'].isin(top_states)]
top_policies_by_state_month = top_states_df.pivot_table(index='state', columns='month', values='policy', aggfunc='count', fill_value=0)
print("Top 3 States Policies Sold by Month:")
print(top_policies_by_state_month)

Top 3 States Policies Sold by Month:
month          1     2
state                 
Arizona     1008   929
California  1918  1634
Oregon      1565  1344


7. The marketing team wants to analyze the effect of different marketing channels on the customer response rate.

Hint: You can use melt to unpivot the data and create a table that shows the customer response rate (those who responded "Yes") by marketing channel.

External Resources for Data Filtering: https://towardsdatascience.com/filtering-data-frames-in-pandas-b570b1f834b9

In [ ]:
response_rate = marketing_df.groupby('sales_channel')['response'].value_counts(normalize=True).unstack().fillna(0) * 100
print("Customer Response Rate by Marketing Channel:")
print(response_rate.round(2))


Customer Response Rate by Marketing Channel:
response          No    Yes
sales_channel              
Agent          80.93  19.07
Branch         88.62  11.38
Call Center    89.02  10.98
Web            88.29  11.71


In [9]:
print(marketing_df.columns)

Index(['unnamed:_0', 'customer', 'state', 'customer_lifetime_value',
       'response', 'coverage', 'education', 'effective_to_date',
       'employmentstatus', 'gender', 'income', 'location_code',
       'marital_status', 'monthly_premium_auto', 'months_since_last_claim',
       'months_since_policy_inception', 'number_of_open_complaints',
       'number_of_policies', 'policy_type', 'policy', 'renew_offer_type',
       'sales_channel', 'total_claim_amount', 'vehicle_class', 'vehicle_size',
       'vehicle_type', 'month'],
      dtype='object')
